In [42]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [43]:
df = pd.read_csv("diabetic_data.csv")

df.replace("?",np.nan,inplace=True)

# df = df.dropna(axis=1, thresh=0.5*len(df))

df.head()

encounter_id  patient_nbr             race  gender      age weight  \
0       2278392      8222157        Caucasian  Female   [0-10)    NaN   
1        149190     55629189        Caucasian  Female  [10-20)    NaN   
2         64410     86047875  AfricanAmerican  Female  [20-30)    NaN   
3        500364     82442376        Caucasian    Male  [30-40)    NaN   
4         16680     42519267        Caucasian    Male  [40-50)    NaN   

   admission_type_id  discharge_disposition_id  admission_source_id  \
0                  6                        25                    1   
1                  1                         1                    7   
2                  1                         1                    7   
3                  1                         1                    7   
4                  1                         1                    7   

   time_in_hospital  ... citoglipton insulin  glyburide-metformin  \
0                 1  ...          No      No                   No   
1                 3  ...          No      Up                   No   
2                 2  ...          No      No                   No   
3                 2  ...          No      Up                   No   
4                 1  ...          No  Steady                   No   

   glipizide-metformin  glimepiride-pioglitazone  metformin-rosiglitazone  \
0                   No                        No                       No   
1                   No                        No                       No   
2                   No                        No                       No   
3                   No                        No                       No   
4                   No                        No                       No   

   metformin-pioglitazone  change diabetesMed readmitted  
0                      No      No          No         NO  
1                      No      Ch         Yes        >30  
2                      No      No         Yes         NO  
3                      No      Ch         Yes         NO  
4                      No      Ch         Yes         NO  

[5 rows x 50 columns]

# 2

In [44]:
from sklearn.preprocessing import LabelEncoder

cat_cols = df.select_dtypes(include='object').columns

le = LabelEncoder()
for col in cat_cols:
    df[col] = le.fit_transform(df[col])


df

encounter_id  patient_nbr  race  gender  age  weight  \
0            2278392      8222157     2       0    0       9   
1             149190     55629189     2       0    1       9   
2              64410     86047875     0       0    2       9   
3             500364     82442376     2       1    3       9   
4              16680     42519267     2       1    4       9   
...              ...          ...   ...     ...  ...     ...   
101761     443847548    100162476     0       1    7       9   
101762     443847782     74694222     0       0    8       9   
101763     443854148     41088789     2       1    7       9   
101764     443857166     31693671     2       0    8       9   
101765     443867222    175429310     2       1    7       9   

        admission_type_id  discharge_disposition_id  admission_source_id  \
0                       6                        25                    1   
1                       1                         1                    7   
2                       1                         1                    7   
3                       1                         1                    7   
4                       1                         1                    7   
...                   ...                       ...                  ...   
101761                  1                         3                    7   
101762                  1                         4                    5   
101763                  1                         1                    7   
101764                  2                         3                    7   
101765                  1                         1                    7   

        time_in_hospital  ...  citoglipton  insulin  glyburide-metformin  \
0                      1  ...            0        1                    1   
1                      3  ...            0        3                    1   
2                      2  ...            0        1                    1   
3                      2  ...            0        3                    1   
4                      1  ...            0        2                    1   
...                  ...  ...          ...      ...                  ...   
101761                 3  ...            0        0                    1   
101762                 5  ...            0        2                    1   
101763                 1  ...            0        0                    1   
101764                10  ...            0        3                    1   
101765                 6  ...            0        1                    1   

        glipizide-metformin  glimepiride-pioglitazone  \
0                         0                         0   
1                         0                         0   
2                         0                         0   
3                         0                         0   
4                         0                         0   
...                     ...                       ...   
101761                    0                         0   
101762                    0                         0   
101763                    0                         0   
101764                    0                         0   
101765                    0                         0   

        metformin-rosiglitazone  metformin-pioglitazone  change  diabetesMed  \
0                             0                       0       1            0   
1                             0                       0       0            1   
2                             0                       0       1            1   
3                             0                       0       0            1   
4                             0                       0       0            1   
...                         ...                     ...     ...          ...   
101761                        0                       0       0            1   
101762                        0                       0       1            1   
101763  

In [45]:
df['readmitted'] = df['readmitted'].apply(lambda x: 1 if x == '<30' else 0)

df['readmitted'].value_counts()


readmitted
0    101766
Name: count, dtype: int64

In [46]:


X = df.drop('readmitted', axis=1)
y = df['readmitted']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


Random Forest

In [47]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

rf_100 = RandomForestClassifier(n_estimators=200,max_leaf_nodes=100,random_state=42)

rf_300 = RandomForestClassifier(n_estimators=200,max_leaf_nodes=300,random_state=42)

rf_100.fit(X_train, y_train)
rf_300.fit(X_train, y_train)

print("RF - 100 leafs")
print(classification_report(y_test, rf_100.predict(X_test)))

print("RF - 300 leafs")
print(classification_report(y_test, rf_300.predict(X_test)))


RF - 100 leafs
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     20354

    accuracy                           1.00     20354
   macro avg       1.00      1.00      1.00     20354
weighted avg       1.00      1.00      1.00     20354

RF - 300 leafs
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     20354

    accuracy                           1.00     20354
   macro avg       1.00      1.00      1.00     20354
weighted avg       1.00      1.00      1.00     20354



Feature Importance و Feature Selection

In [48]:
importances = rf_300.feature_importances_
feat_imp = pd.Series(importances, index=X.columns).sort_values(ascending=False)

feat_imp.head(10)


encounter_id                0.0
patient_nbr                 0.0
race                        0.0
gender                      0.0
age                         0.0
weight                      0.0
admission_type_id           0.0
discharge_disposition_id    0.0
admission_source_id         0.0
time_in_hospital            0.0
dtype: float64

In [49]:
#انتخاب مهم‌ترین فیچرها
top_features = feat_imp.head(10).index

X_train_top = X_train[top_features]
X_test_top = X_test[top_features]



XGBoost

In [51]:
from xgboost import XGBClassifier

xgb = XGBClassifier(
    n_estimators=200,
    max_depth=5,
    learning_rate=0.1,
    scale_pos_weight = y_train.value_counts()[0],
    random_state=42
)

xgb.fit(X_train_top, y_train)

print(classification_report(y_test, xgb.predict(X_test_top)))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00     20354

    accuracy                           1.00     20354
   macro avg       1.00      1.00      1.00     20354
weighted avg       1.00      1.00      1.00     20354

